# Chapter 5: Reading Data

In [1368]:
%run -i 'ch03.ipynb'

2024-03-21 10:46:57,589 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-21 10:46:57,589 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2024-03-21 10:46:57,590 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-21 10:46:57,591 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2024-03-21 10:46:57,591 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-21 10:46:57,591 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customer")
2024-03-21 10:46:57,592 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-21 10:46:57,592 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order")
2024-03-21 10:46:57,593 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-21 10:46:57,593 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order_detail")
2024-03-21 10:46:57,594 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-21 10:46:57,594 INFO sqlalchemy.engine.Engine COMMIT


In [1369]:
DATABASE_URL

'sqlite+pysqlite:///store.db'

In [1370]:
conn = engine.connect()

Necessary imports:

In [1371]:
from typing import Sequence

from sqlalchemy import (Connection, Row, String, and_, asc, between, cast,
                        distinct, func, not_, or_, select, text)

Target `Table`:

In [1372]:
employee

Table('employee', MetaData(), Column('employee_id', Integer(), table=<employee>, primary_key=True, nullable=False), Column('manager_id', Integer(), ForeignKey('employee.employee_id'), table=<employee>), Column('name', String(length=31), table=<employee>, nullable=False), Column('is_manager', Boolean(), table=<employee>, default=ScalarElementColumnDefault(False)), Column('hire_date', Date(), table=<employee>, default=CallableColumnDefault(<function date.today at 0x7f8331a7e8c0>)), schema=None)

A `Select` construct:

In [1373]:
stmt = (
    select(employee)
)

In [1374]:
stmt

In [1375]:
print(stmt)

SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee


In [1376]:
result = conn.execute(stmt)

2024-03-21 10:46:57,652 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-21 10:46:57,652 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee
2024-03-21 10:46:57,653 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ()


In [1377]:
result

`rowcount` is not supported for SQLite 3 (you can use PostgreSQL as recommended):

As required by the Python DB API Spec, the rowcount attribute “is -1 in case no executeXX() has been performed on the cursor or the rowcount of the last operation is not determinable by the interface”.
This includes SELECT statements because we cannot determine the number of rows a query produced until all rows were fetched.

In [1378]:
result.rowcount

-1

You can still get the count of rows with:

In [1379]:
employee_rows = result.all()
print(len(employee_rows))

6


In [1380]:
for row in employee_rows:
    print(row)

(1, None, 'Alice', True, datetime.date(2024, 3, 20))
(2, 1, 'Bob', False, datetime.date(2024, 3, 21))
(3, 1, 'Cathy', False, datetime.date(2024, 3, 21))
(4, None, 'Louis', True, datetime.date(2024, 2, 20))
(5, 4, 'Lilly', False, datetime.date(2024, 3, 1))
(6, None, 'Alice', False, datetime.date(2024, 3, 21))


Limiting returned columns:

In [1381]:
stmt = (
    select(employee.c.name, employee.columns.hire_date)
)

In [1382]:
print(stmt)

SELECT employee.name, employee.hire_date 
FROM employee


In [1383]:
result = conn.execute(stmt)
for row in result:
    print(row)

2024-03-21 10:46:57,703 INFO sqlalchemy.engine.Engine SELECT employee.name, employee.hire_date 
FROM employee
2024-03-21 10:46:57,704 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ()
('Alice', datetime.date(2024, 3, 20))
('Bob', datetime.date(2024, 3, 21))
('Cathy', datetime.date(2024, 3, 21))
('Louis', datetime.date(2024, 2, 20))
('Lilly', datetime.date(2024, 3, 1))
('Alice', datetime.date(2024, 3, 21))


Sorting the result set:

In [1384]:
stmt = (
    select(employee.c.name, employee.c.hire_date)
    .order_by(asc(employee.c.hire_date))
)

In [1385]:
print(stmt)

SELECT employee.name, employee.hire_date 
FROM employee ORDER BY employee.hire_date ASC


In [1386]:
result = conn.execute(stmt)
for row in result:
    print(row)

2024-03-21 10:46:57,722 INFO sqlalchemy.engine.Engine SELECT employee.name, employee.hire_date 
FROM employee ORDER BY employee.hire_date ASC
2024-03-21 10:46:57,722 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ()
('Louis', datetime.date(2024, 2, 20))
('Lilly', datetime.date(2024, 3, 1))
('Alice', datetime.date(2024, 3, 20))
('Bob', datetime.date(2024, 3, 21))
('Cathy', datetime.date(2024, 3, 21))
('Alice', datetime.date(2024, 3, 21))


Limiting the result set:

In [1387]:
num = 3

stmt = (
    select(employee.c.name, employee.c.hire_date)
    .order_by(asc(employee.c.hire_date))
    .limit(num)
)

In [1388]:
print(stmt)

SELECT employee.name, employee.hire_date 
FROM employee ORDER BY employee.hire_date ASC
 LIMIT :param_1


In [1389]:
result = conn.execute(stmt)
for row in result:
    print(row)

2024-03-21 10:46:57,739 INFO sqlalchemy.engine.Engine SELECT employee.name, employee.hire_date 
FROM employee ORDER BY employee.hire_date ASC
 LIMIT ? OFFSET ?
2024-03-21 10:46:57,739 INFO sqlalchemy.engine.Engine [generated in 0.00059s] (3, 0)
('Louis', datetime.date(2024, 2, 20))
('Lilly', datetime.date(2024, 3, 1))
('Alice', datetime.date(2024, 3, 20))


Labeling columns:

In [1390]:
stmt = (
    select(
        employee.c.name,
        ("hire date: " + cast(employee.c.hire_date, String))
        .label("hired_on")
    )
)

In [1391]:
print(stmt)

SELECT employee.name, :param_1 || CAST(employee.hire_date AS VARCHAR) AS hired_on 
FROM employee


In [1392]:
result = conn.execute(stmt)

2024-03-21 10:46:57,752 INFO sqlalchemy.engine.Engine SELECT employee.name, ? || CAST(employee.hire_date AS VARCHAR) AS hired_on 
FROM employee
2024-03-21 10:46:57,753 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ('hire date: ',)


In [1393]:
print("keys:", result.keys())

keys: RMKeyView(['name', 'hired_on'])


In [1394]:
for row in result:
    print(f"{row.name:>5}, {row.hired_on}")

Alice, hire date: 2024-03-20
  Bob, hire date: 2024-03-21
Cathy, hire date: 2024-03-21
Louis, hire date: 2024-02-20
Lilly, hire date: 2024-03-01
Alice, hire date: 2024-03-21


Filtering:

In [1395]:
stmt = (
    select(employee)
    .where(employee.c.is_manager == True)
)

In [1396]:
print(stmt)

SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = true


In [1397]:
result = conn.execute(stmt)
for row in result:
    print(row)

2024-03-21 10:46:57,772 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = 1
2024-03-21 10:46:57,773 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ()
(1, None, 'Alice', True, datetime.date(2024, 3, 20))
(4, None, 'Louis', True, datetime.date(2024, 2, 20))


Multiple filters:

In [1398]:
stmt = (
    select(employee)
    .where(employee.c.is_manager == True)
    .where(employee.c.name == "Alice")
)

In [1399]:
print(stmt)

SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = true AND employee.name = :name_1


In [1400]:
result = conn.execute(stmt)
for row in result:
    print(row)

2024-03-21 10:46:57,787 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = 1 AND employee.name = ?
2024-03-21 10:46:57,787 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ('Alice',)
(1, None, 'Alice', True, datetime.date(2024, 3, 20))


Another way to write this would be:

In [1401]:
stmt = (
    select(employee)
    .where(
        employee.c.is_manager == True,
        employee.c.name == "Alice",
    )
)

In [1402]:
print(stmt)

SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = true AND employee.name = :name_1


Conjunctions: AND, OR

In [1403]:
from sqlalchemy import and_, or_

Rewrite the previous example using AND:

In [1404]:
stmt = (
    select(employee)
    .where(
        and_(employee.c.is_manager == True,
              employee.c.name == "Alice",)
    )
)

In [1405]:
print(stmt)

SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = true AND employee.name = :name_1


Using OR:

In [1406]:
stmt = (
    select(employee)
    .where(
        and_(
            employee.c.is_manager == True,
            or_(
                employee.c.name == "Alice",
                employee.c.name == "Louis",
            )
        )
    )
)

In [1407]:
print(stmt)

SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = true AND (employee.name = :name_1 OR employee.name = :name_2)


In [1408]:
result = conn.execute(stmt)
for row in result:
    print(row)

2024-03-21 10:46:57,823 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = 1 AND (employee.name = ? OR employee.name = ?)
2024-03-21 10:46:57,824 INFO sqlalchemy.engine.Engine [generated in 0.00092s] ('Alice', 'Louis')
(1, None, 'Alice', True, datetime.date(2024, 3, 20))
(4, None, 'Louis', True, datetime.date(2024, 2, 20))


Negation: NOT

In [1409]:
stmt = (
    select(employee)
    .where(
        and_(
            employee.c.is_manager == True,
            not_(employee.c.name == "Alice"),
        )
    )
)

In [1410]:
print(stmt)

SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = true AND employee.name != :name_1


In [1411]:
stmt.compile().params

{'name_1': 'Alice'}

In [1412]:
result = conn.execute(stmt)
for row in result:
    print(row)

2024-03-21 10:46:57,842 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.is_manager = 1 AND employee.name != ?
2024-03-21 10:46:57,842 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ('Alice',)
(4, None, 'Louis', True, datetime.date(2024, 2, 20))


Searching for "Alice"

In [1413]:
def get_all_alices(conn: Connection):
    stmt = (
        select(employee)
        .where(
            employee.c.name == "Alice"
        )
    )
    result = conn.execute(stmt)

    return result.all()

for row in get_all_alices(conn):
    print(row)

2024-03-21 10:46:57,848 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE employee.name = ?
2024-03-21 10:46:57,849 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ('Alice',)
(1, None, 'Alice', True, datetime.date(2024, 3, 20))
(6, None, 'Alice', False, datetime.date(2024, 3, 21))


BETWEEN (prices between 10 and 100):

In [1414]:
stmt = (
    select(
        product.c.product_name,
        product.c.unit_price,
    )
    .where(
        product.c.unit_price.between(10, 100)
    )
)

In [1415]:
print(stmt)

SELECT product.product_name, product.unit_price 
FROM product 
WHERE product.unit_price BETWEEN :unit_price_1 AND :unit_price_2


In [1416]:
print(stmt.compile().params)

{'unit_price_1': 10, 'unit_price_2': 100}


In [1417]:
result = conn.execute(stmt)
for row in result:
    print(f"{row[0]:<18}: ${float(row[1])}")

2024-03-21 10:46:57,868 INFO sqlalchemy.engine.Engine SELECT product.product_name, product.unit_price 
FROM product 
WHERE product.unit_price BETWEEN ? AND ?
2024-03-21 10:46:57,869 INFO sqlalchemy.engine.Engine [generated in 0.00092s] (10, 100)
headphone         : $25.99
digital camera    : $45.99
memory card 256GB : $21.99


DISTINCT:

In [1418]:
stmt = (
    select(employee.c.name.distinct())
    .order_by(employee.c.name)
)

In [1419]:
print(stmt)

SELECT DISTINCT employee.name 
FROM employee ORDER BY employee.name


In [1420]:
result = conn.scalars(stmt)

2024-03-21 10:46:57,882 INFO sqlalchemy.engine.Engine SELECT DISTINCT employee.name 
FROM employee ORDER BY employee.name
2024-03-21 10:46:57,883 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ()


In [1421]:
for name in result:
    print(name)

Alice
Bob
Cathy
Lilly
Louis


IN:

In [1422]:
columns = [product.c.product_name, product.c.type]
stmt = (
    select(*columns)
    .where(product.c.type.in_([ProductType.PHONE, ProductType.ACCESSORY]))
)

In [1423]:
print(stmt)

SELECT product.product_name, product.type 
FROM product 
WHERE product.type IN (__[POSTCOMPILE_type_1])


In [1424]:
print(stmt.compile().params)

{'type_1': [<ProductType.PHONE: 0>, <ProductType.ACCESSORY: 1>]}


In [1425]:
for row in conn.execute(stmt):
    print(f"{(row[1].name):<9}: {row.product_name}")

2024-03-21 10:46:57,906 INFO sqlalchemy.engine.Engine SELECT product.product_name, product.type 
FROM product 
WHERE product.type IN (?, ?)
2024-03-21 10:46:57,906 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ('PHONE', 'ACCESSORY')
PHONE    : phone
ACCESSORY: phone screen protector
ACCESSORY: headphone
ACCESSORY: memory card 256GB


LIKE/ILIKE:

In [1426]:
stmt = (
    select(product.c.product_name)
    .where(product.c.product_name.ilike("%phone%"))
)

In [1427]:
print(stmt)

SELECT product.product_name 
FROM product 
WHERE lower(product.product_name) LIKE lower(:product_name_1)


In [1428]:
print(stmt.compile().params)

{'product_name_1': '%phone%'}


In [1429]:
for product_name in conn.scalars(stmt):
    print(product_name)

2024-03-21 10:46:57,925 INFO sqlalchemy.engine.Engine SELECT product.product_name 
FROM product 
WHERE lower(product.product_name) LIKE lower(?)
2024-03-21 10:46:57,925 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ('%phone%',)
phone
phone screen protector
headphone


Joins:

In [1430]:
order_id = 1

In [1431]:
columns = [
    product.c.product_name,
    product.c.unit_price,
    order_detail.c.quantity,
    order.c.order_datetime,
]

stmt = (
    select(*columns)
    .select_from(order_detail)
    .join(order)
    .join(product)
    .where(order.c.order_id == order_id)
)

result = conn.execute(stmt)

for row in result:
    print(
        f'{row.order_datetime.strftime("%Y-%m-%d %H:%M:%S")} '
        f'{row.product_name:<22}: '
        f'{row.unit_price:>7} *{row.quantity}'
    )

2024-03-21 10:46:57,939 INFO sqlalchemy.engine.Engine SELECT product.product_name, product.unit_price, order_detail.quantity, "order".order_datetime 
FROM order_detail JOIN "order" ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".order_id = ?
2024-03-21 10:46:57,939 INFO sqlalchemy.engine.Engine [generated in 0.00095s] (1,)
2024-03-21 10:46:44 phone                 :  300.00 *1
2024-03-21 10:46:44 phone screen protector:    9.50 *1
2024-03-21 10:46:44 headphone             :   25.99 *1


In [1432]:
print(stmt)

SELECT product.product_name, product.unit_price, order_detail.quantity, "order".order_datetime 
FROM order_detail JOIN "order" ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".order_id = :order_id_1


You can also rewrite the above `Select` construct with `Select.join_from()`:

In [1433]:
stmt = (
    select(*columns)
    .join_from(order, order_detail)
    .join_from(order_detail, product)
    .where(order.c.order_id == order_id)
)

In [1434]:
print(stmt)

SELECT product.product_name, product.unit_price, order_detail.quantity, "order".order_datetime 
FROM "order" JOIN order_detail ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".order_id = :order_id_1


To specify the ON clause explicitly:

In [1435]:
stmt = (
    select(*columns)
    .select_from(order_detail)
    .join(order, order_detail.c.order_id == order.c.order_id)
    .join(product, order_detail.c.product_id == product.c.product_id)
    .where(order.c.order_id == order_id)
)

In [1436]:
print(stmt)

SELECT product.product_name, product.unit_price, order_detail.quantity, "order".order_datetime 
FROM order_detail JOIN "order" ON order_detail.order_id = "order".order_id JOIN product ON order_detail.product_id = product.product_id 
WHERE "order".order_id = :order_id_1


Aliases and Self-Joins:

In [1437]:
manager = employee.alias("manager")

columns = [
    employee.c.employee_id,
    employee.c.name,
    manager.c.name.label("manager_name"),
    manager.c.employee_id.label("manager_id"),
]

stmt = (
    select(*columns)
    .select_from(employee)
    .join(
        manager,
        employee.c.manager_id == manager.c.employee_id,
        isouter=True,
    )
)

result = conn.execute(stmt)
for row in result:
    if row.manager_name is not None:
        print(f"{row.name}({row.employee_id})'s manager is "
                f"{row.manager_name}({row.manager_id}).")
    else:
        print(f"{row.name}({row.employee_id}) has no manager.")

2024-03-21 10:46:57,973 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.name, manager.name AS manager_name, manager.employee_id AS manager_id 
FROM employee LEFT OUTER JOIN employee AS manager ON employee.manager_id = manager.employee_id
2024-03-21 10:46:57,974 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ()
Alice(1) has no manager.
Bob(2)'s manager is Alice(1).
Cathy(3)'s manager is Alice(1).
Louis(4) has no manager.
Lilly(5)'s manager is Louis(4).
Alice(6) has no manager.


In [1438]:
print(stmt)

SELECT employee.employee_id, employee.name, manager.name AS manager_name, manager.employee_id AS manager_id 
FROM employee LEFT OUTER JOIN employee AS manager ON employee.manager_id = manager.employee_id


SQL Functions:

COUNT:

In [1439]:
stmt = (
    select(func.count())
    .select_from(employee)
)

In [1440]:
print(stmt)

SELECT count(*) AS count_1 
FROM employee


In [1441]:
count = conn.scalar(stmt)
print(f"We have {count} employees.")

2024-03-21 10:46:57,992 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM employee
2024-03-21 10:46:57,993 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ()
We have 6 employees.


MAX:

In [1442]:
stmt = (
    select(func.max(product.c.unit_price))
)

In [1443]:
print(stmt)

SELECT max(product.unit_price) AS max_1 
FROM product


In [1444]:
max_price = conn.scalar(stmt)
print(f"The most expensive product costs ${max_price}.")

2024-03-21 10:46:58,008 INFO sqlalchemy.engine.Engine SELECT max(product.unit_price) AS max_1 
FROM product
2024-03-21 10:46:58,009 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ()
The most expensive product costs $300.00.


GROUP BY, HAVING:

In [1445]:
def list_all_managers_with_employee_count(
        conn: Connection,
        max_subordinates=4,
):
    manager = employee.alias("manager")
    stmt = (
        select(
            manager.c.name,
            func.count(employee.c.employee_id).label("count"),
        )
        .join(employee, employee.c.manager_id == manager.c.employee_id)
        .group_by(manager.c.employee_id)
        .having(func.count(employee.c.employee_id) < max_subordinates)
        .order_by(func.count(employee.c.employee_id))
    )

    result = conn.execute(stmt)
    for row in result:
        print(f"{row.name}: {row.count}")

In [1446]:
list_all_managers_with_employee_count(conn)

2024-03-21 10:46:58,021 INFO sqlalchemy.engine.Engine SELECT manager.name, count(employee.employee_id) AS count 
FROM employee AS manager JOIN employee ON employee.manager_id = manager.employee_id GROUP BY manager.employee_id 
HAVING count(employee.employee_id) < ? ORDER BY count(employee.employee_id)
2024-03-21 10:46:58,022 INFO sqlalchemy.engine.Engine [generated in 0.00116s] (4,)
Louis: 1
Alice: 2


In [1447]:
list_all_managers_with_employee_count(conn, 2)

2024-03-21 10:46:58,028 INFO sqlalchemy.engine.Engine SELECT manager.name, count(employee.employee_id) AS count 
FROM employee AS manager JOIN employee ON employee.manager_id = manager.employee_id GROUP BY manager.employee_id 
HAVING count(employee.employee_id) < ? ORDER BY count(employee.employee_id)
2024-03-21 10:46:58,029 INFO sqlalchemy.engine.Engine [cached since 0.007531s ago] (2,)
Louis: 1


Raw SQL Queries:

In [1448]:
t = text("SELECT customer_id, first_name FROM customer")
result = conn.execute(t)
for row in result:
    print(row.customer_id, row.first_name)

2024-03-21 10:46:58,035 INFO sqlalchemy.engine.Engine SELECT customer_id, first_name FROM customer
2024-03-21 10:46:58,036 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ()
1 Alex
2 Mary


Using bind parameters:

In [1449]:
customer_id = 1

t = (
    text(
        "SELECT customer_id, first_name "
        "FROM customer "
        "WHERE customer_id=:id"
    )
    .bindparams(id=customer_id)
)

In [1450]:
result = conn.execute(t).first()
print(result)

2024-03-21 10:46:58,046 INFO sqlalchemy.engine.Engine SELECT customer_id, first_name FROM customer WHERE customer_id=?
2024-03-21 10:46:58,047 INFO sqlalchemy.engine.Engine [generated in 0.00080s] (1,)
(1, 'Alex')


In [1451]:
t = (
    select(customer.c.customer_id, customer.c.first_name)
    .where(
        text("customer_id=:id")
        .bindparams(id=customer_id)
    )
)

In [1452]:
print(t)

SELECT customer.customer_id, customer.first_name 
FROM customer 
WHERE customer_id=:id


Constructing Conditional Queries:

In [1453]:
def get_orders_for_customers(
        conn: Connection,
        name: str | None = None,
        is_shipped: bool = False,
        details: bool = True
) -> Sequence[Row]:
    print(f"> Params: name={name}, is_shipped={is_shipped}, details={details}")

    # put columns in an individual array, so we can extend it later
    columns = [
        customer.c.first_name,
        customer.c.last_name,
        order.c.order_id,
        order.c.is_shipped,
    ]
    # these two tables are always needed, so join them first
    joined = customer.join(order)

    # if details are required, more columns and tables are incorporated
    if details:
        columns.extend([
            product.c.product_name,
            order_detail.c.quantity,
            product.c.unit_price,
        ])
        joined = joined.join(order_detail).join(product)

    # the rows are filtered by the shipping status
    stmt = (
        select(*columns)
        .select_from(joined)
        .where(order.c.is_shipped == is_shipped)
    )

    # and also filtered by first name
    if name is not None:
        stmt = stmt.where(customer.c.first_name == name)

    # print("SQL:", stmt)

    result = conn.execute(stmt)

    return result.all()

In [1454]:
def print_orders_for_customers(orders: Sequence[Row]) -> None:
    order_id = None
    for row in orders:
        current_order_id = row.order_id
        if order_id is None or order_id != current_order_id:
            order_id = current_order_id
            print(f"{row.first_name} {row.last_name} "
                  f"has an order with ID:{current_order_id}")
        if hasattr(row, "product_name"):
            print(f"- {row.product_name}: ${row.unit_price} *{row.quantity}")

Display all orders that have not been shipped, including details:

In [1455]:
orders = get_orders_for_customers(
    conn, None, is_shipped=False, details=True,
)
print_orders_for_customers(orders)

> Params: name=None, is_shipped=False, details=True
2024-03-21 10:46:58,074 INFO sqlalchemy.engine.Engine SELECT customer.first_name, customer.last_name, "order".order_id, "order".is_shipped, product.product_name, order_detail.quantity, product.unit_price 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id JOIN order_detail ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".is_shipped = 0
2024-03-21 10:46:58,075 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ()
Alex Smith has an order with ID:1
- phone: $300.00 *1
- phone screen protector: $9.50 *1
- headphone: $25.99 *1
Alex Smith has an order with ID:2
- memory card 256GB: $21.99 *1
Mary Taylor has an order with ID:3
- digital camera: $45.99 *1
- memory card 256GB: $21.99 *2


Display orders for Alex that have not been shipped, including details:

In [1456]:
orders = get_orders_for_customers(
    conn, "Alex", is_shipped=False, details=True,
)
print_orders_for_customers(orders)

> Params: name=Alex, is_shipped=False, details=True
2024-03-21 10:46:58,081 INFO sqlalchemy.engine.Engine SELECT customer.first_name, customer.last_name, "order".order_id, "order".is_shipped, product.product_name, order_detail.quantity, product.unit_price 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id JOIN order_detail ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".is_shipped = 0 AND customer.first_name = ?
2024-03-21 10:46:58,081 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ('Alex',)
Alex Smith has an order with ID:1
- phone: $300.00 *1
- phone screen protector: $9.50 *1
- headphone: $25.99 *1
Alex Smith has an order with ID:2
- memory card 256GB: $21.99 *1


Repeat, this time without including details:

In [1457]:
orders = get_orders_for_customers(
    conn, "Alex", is_shipped=False, details=False,
)
print_orders_for_customers(orders)

> Params: name=Alex, is_shipped=False, details=False
2024-03-21 10:46:58,087 INFO sqlalchemy.engine.Engine SELECT customer.first_name, customer.last_name, "order".order_id, "order".is_shipped 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id 
WHERE "order".is_shipped = 0 AND customer.first_name = ?
2024-03-21 10:46:58,088 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ('Alex',)
Alex Smith has an order with ID:1
Alex Smith has an order with ID:2


Finally, display all shipped orders for Alex, excluding details (this should be empty):

In [1458]:
orders = get_orders_for_customers(
    conn, "Alex", is_shipped=True, details=False,
)
print_orders_for_customers(orders)

> Params: name=Alex, is_shipped=True, details=False
2024-03-21 10:46:58,094 INFO sqlalchemy.engine.Engine SELECT customer.first_name, customer.last_name, "order".order_id, "order".is_shipped 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id 
WHERE "order".is_shipped = 1 AND customer.first_name = ?
2024-03-21 10:46:58,095 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ('Alex',)


Close the connection after you're done:

In [1459]:
conn.close()

2024-03-21 10:46:58,100 INFO sqlalchemy.engine.Engine ROLLBACK
